# 数据集转换

## coseal/Magicoder-Evol-Instruct-110K-sft

In [14]:
from datasets import load_dataset

dataset = load_dataset("coseal/Magicoder-Evol-Instruct-110K-sft")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 100772
    })
})


In [15]:
import pandas as pd

df = pd.DataFrame(dataset["train"])
df

,instruction,response
0,Please amend the subsequent Python script so t...,```python\n# Establish an integer list\narr = ...
1,"i've got this python code from an ocr tool, bu...",There are several issues with your code like f...
2,Create a recursive function in Java that predi...,You can achieve this using a combination of re...
3,Develop a program that uses natural language p...,This task requires writing of a significant vo...
4,I am trying to create a load testing script fo...,The correct approach would depend on the preci...
...,...,...
100767,Create a function that not only detects any ch...,"In Python, we could use the difflib module and..."
100768,Conduct a comprehensive exploratory data analy...,"Sorry, as an AI developed by Open AI, I can't ..."
100769,"I used OCR to get this Python code, but it's f...","Sure, I can help with that. Let me reformat th..."
100770,Write a JavaScript function that will take an ...,"Sure, I'll provide you with a simple example o..."


In [16]:
from metagpt.utils.common import OutputParser

def is_python_code(row):
    # instruction = row.instruction
    response = row.response

    try:
        code = OutputParser.parse_code(response, "python")
        if len(code) > 10:
            return True
    except:
        pass
    return False

df["ispy"] = df.apply(is_python_code, axis=1)
xdf = df[df.ispy][["instruction","response"]].copy().rename(columns={"instruction": "input", "response": "output"})
xdf

2024-03-26 19:23:32.328 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to /Users/deryzhou/Downloads/StreamChatPlayground/notebooks


,input,output
0,Please amend the subsequent Python script so t...,```python\n# Establish an integer list\narr = ...
1,"i've got this python code from an ocr tool, bu...",There are several issues with your code like f...
3,Develop a program that uses natural language p...,This task requires writing of a significant vo...
8,Develop a method that not only incorporates a ...,You can implement this task in Python:\n\n```p...
9,Write a Python program that takes a list and a...,Here is a Python program that does just that:\...
...,...,...
100763,Modify and optimize the given erroneous code t...,This erroneous code uses the bubble sort algor...
100764,Utilize the BeautifulSoup package to orchestra...,You can extract the title tag content from the...
100765,Implement a function that can insert multiple ...,Here is the Python implementation:\n\n```pytho...
100767,Create a function that not only detects any ch...,"In Python, we could use the difflib module and..."


In [17]:
xdf.to_json("magicoder.jsonl", orient='records', lines=True)

# 数据收集

## SWE-bench

In [22]:
import sys
sys.path.append("..")

from SWE_bench.inference.make_datasets.create_instance import PROMPT_FUNCTIONS, add_text_inputs, TOKENIZER_FUNCS

PROMPT_FUNCTIONS

{'style-2': <function SWE_bench.inference.make_datasets.create_instance.prompt_style_2(instance)>,
 'style-3': <function SWE_bench.inference.make_datasets.create_instance.prompt_style_3(instance)>,
 'full_file_gen': <function SWE_bench.inference.make_datasets.create_instance.full_file_gen(instance)>,
 'style-2-edits-only': <function SWE_bench.inference.make_datasets.create_instance.prompt_style_2_edits_only(instance)>}

In [18]:
from datasets import load_dataset

dataset = load_dataset("princeton-nlp/SWE-bench")
print(dataset)

Generating dev split:   0%|          | 0/225 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2294 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/19008 [00:00<?, ? examples/s]

DatasetDict({
    dev: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
        num_rows: 225
    })
    test: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
        num_rows: 2294
    })
    train: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
        num_rows: 19008
    })
})


In [21]:
import pandas as pd

df = pd.DataFrame(dataset["dev"])
df.iloc[0]

repo                                                        sqlfluff/sqlfluff
instance_id                                           sqlfluff__sqlfluff-4764
base_commit                          a820c139ccbe6d1865d73c4a459945cd69899f8f
patch                       diff --git a/src/sqlfluff/cli/commands.py b/sr...
test_patch                  diff --git a/test/cli/commands_test.py b/test/...
problem_statement           Enable quiet mode/no-verbose in CLI for use in...
hints_text                                                                   
created_at                                               2023-04-16T14:24:42Z
version                                                                   1.4
FAIL_TO_PASS                ["test/cli/commands_test.py::test__cli__fix_mu...
PASS_TO_PASS                ["test/cli/commands_test.py::test__cli__comman...
environment_setup_commit             d19de0ecd16d298f9e3bfb91da122734c40c01e5
Name: 0, dtype: object

In [28]:
from copy import deepcopy

instance = deepcopy(df.iloc[0])
instance["readmes"] = dict()
instance["file_contents"] = dict()
base_text_inputs = PROMPT_FUNCTIONS["style-3"](instance)
print(base_text_inputs)

You will be provided with a partial code base and an issue statement explaining a problem to resolve.
<issue>
Enable quiet mode/no-verbose in CLI for use in pre-commit hook
There seems to be only an option to increase the level of verbosity when using SQLFluff [CLI](https://docs.sqlfluff.com/en/stable/cli.html), not to limit it further.

It would be great to have an option to further limit the amount of prints when running `sqlfluff fix`, especially in combination with deployment using a pre-commit hook. For example, only print the return status and the number of fixes applied, similar to how it is when using `black` in a pre-commit hook:
![image](https://user-images.githubusercontent.com/10177212/140480676-dc98d00b-4383-44f2-bb90-3301a6eedec2.png)

This hides the potentially long list of fixes that are being applied to the SQL files, which can get quite verbose.

</issue>

<code>


</code>

Here is an example of a patch file. It consists of changes to the code base. It specifies the f

In [30]:
tokenizer, tokenizer_func = TOKENIZER_FUNCS["cl100k"]
len(tokenizer_func(base_text_inputs, tokenizer))

738